In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from catboost import CatBoostClassifier

from skopt import BayesSearchCV

from xgboost import XGBClassifier

%matplotlib inline

In [44]:
class BayesSearchCV(BayesSearchCV):
    def _run_search(self, x): raise BaseException('Use newer skopt')

In [2]:
df = pd.read_csv('./data/model12218.csv', index_col = 0)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64339 entries, 0 to 75010
Data columns (total 18 columns):
age                 64339 non-null int64
animal_type         64339 non-null object
dow                 64339 non-null int64
fixed_status        64339 non-null object
gender              64339 non-null object
group               64339 non-null object
intake_condition    64339 non-null object
intake_season       64339 non-null object
intake_type         64339 non-null object
mix                 64339 non-null int64
simple_color        64339 non-null object
hour_in             64339 non-null int64
simple_group        64339 non-null object
shade               64339 non-null object
given_name          47018 non-null object
name_freq           64339 non-null int64
days_in_shelter     64339 non-null int64
outcome_type        64339 non-null object
dtypes: int64(6), object(12)
memory usage: 9.3+ MB


In [5]:
df['gender'] = df['gender'].astype(str)

In [8]:
df['dow'] = df['dow'].astype(str)
df['mix'] = df['mix'].astype(str)
df['hour_in'] = df['hour_in'].astype(str)

In [11]:
df = df.drop(labels=['days_in_shelter'], axis=1)

In [14]:
df = df[~df.duplicated()]

In [15]:
df.reset_index(inplace=True, drop=True)

In [20]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [29]:
X = X.drop(labels=['given_name'], axis=1)

In [ ]:
le = LabelEncoder()
le.fit(y)
y = le.transform(y)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [40]:
X.head()

,age,animal_type,dow,fixed_status,gender,group,intake_condition,intake_season,intake_type,mix,simple_color,hour_in,simple_group,shade,name_freq
0,2920,Dog,6,fixed,1,Sporting,Normal,summer,Stray,0,White,12,Sporting,Light,52
1,330,Dog,3,intact,0,Hound,Normal,spring,Stray,1,Sable,18,Non-Sporting,Light,1
2,1460,Dog,6,fixed,0,Working,Normal,spring,Stray,1,Yellow,10,Non-Sporting,Light,22
3,730,Dog,5,fixed,0,Sporting,Normal,winter,Owner Surrender,1,Brown,12,Sporting,Medium,32
4,730,Dog,3,intact,0,Sporting,Normal,summer,Public Assist,1,Black,14,Sporting,Dark,1


In [53]:
cat = CatBoostClassifier(cat_features = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])

tuning_params = {'one_hot_max_size': [24], 'learning_rate': [1e-2, 1], 'loss_function': ['MultiClass'],
                 'depth': [6, 10], 'n_estimators': [800]}


bs = BayesSearchCV(cat, tuning_params, cv=3, verbose=True, scoring='neg_log_loss', n_jobs=-1, n_iter=10)

bs.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 221.0min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  3.6min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

309:	learn: -0.8627793	total: 41.6s	remaining: 1m 5s
310:	learn: -0.8626684	total: 41.7s	remaining: 1m 5s
311:	learn: -0.8624559	total: 41.9s	remaining: 1m 5s
312:	learn: -0.8622844	total: 42s	remaining: 1m 5s
313:	learn: -0.8621273	total: 42.2s	remaining: 1m 5s
314:	learn: -0.8620309	total: 42.3s	remaining: 1m 5s
315:	learn: -0.8619549	total: 42.4s	remaining: 1m 4s
316:	learn: -0.8619037	total: 42.5s	remaining: 1m 4s
317:	learn: -0.8617289	total: 42.6s	remaining: 1m 4s
318:	learn: -0.8616206	total: 42.8s	remaining: 1m 4s
319:	learn: -0.8615474	total: 42.9s	remaining: 1m 4s
320:	learn: -0.8614487	total: 43.1s	remaining: 1m 4s
321:	learn: -0.8613791	total: 43.2s	remaining: 1m 4s
322:	learn: -0.8612374	total: 43.4s	remaining: 1m 4s
323:	learn: -0.8611377	total: 43.5s	remaining: 1m 3s
324:	learn: -0.8610079	total: 43.6s	remaining: 1m 3s
325:	learn: -0.8609306	total: 43.8s	remaining: 1m 3s
326:	learn: -0.8608145	total: 43.9s	remaining: 1m 3s
327:	learn: -0.8607382	total: 44.1s	remaining: 1

467:	learn: -0.8503663	total: 1m 3s	remaining: 45.2s
468:	learn: -0.8503374	total: 1m 3s	remaining: 45s
469:	learn: -0.8503120	total: 1m 3s	remaining: 44.9s
470:	learn: -0.8502146	total: 1m 4s	remaining: 44.8s
471:	learn: -0.8501909	total: 1m 4s	remaining: 44.6s
472:	learn: -0.8501459	total: 1m 4s	remaining: 44.5s
473:	learn: -0.8501259	total: 1m 4s	remaining: 44.3s
474:	learn: -0.8500890	total: 1m 4s	remaining: 44.2s
475:	learn: -0.8500053	total: 1m 4s	remaining: 44s
476:	learn: -0.8498777	total: 1m 4s	remaining: 43.9s
477:	learn: -0.8498129	total: 1m 4s	remaining: 43.8s
478:	learn: -0.8496973	total: 1m 5s	remaining: 43.7s
479:	learn: -0.8496307	total: 1m 5s	remaining: 43.5s
480:	learn: -0.8495427	total: 1m 5s	remaining: 43.4s
481:	learn: -0.8493988	total: 1m 5s	remaining: 43.3s
482:	learn: -0.8492228	total: 1m 5s	remaining: 43.2s
483:	learn: -0.8491927	total: 1m 6s	remaining: 43.1s
484:	learn: -0.8491495	total: 1m 6s	remaining: 43s
485:	learn: -0.8491230	total: 1m 6s	remaining: 42.8s

621:	learn: -0.8405815	total: 1m 24s	remaining: 24.3s
622:	learn: -0.8405726	total: 1m 25s	remaining: 24.2s
623:	learn: -0.8404650	total: 1m 25s	remaining: 24s
624:	learn: -0.8403579	total: 1m 25s	remaining: 23.9s
625:	learn: -0.8402878	total: 1m 25s	remaining: 23.8s
626:	learn: -0.8402379	total: 1m 25s	remaining: 23.6s
627:	learn: -0.8401935	total: 1m 25s	remaining: 23.5s
628:	learn: -0.8401028	total: 1m 25s	remaining: 23.4s
629:	learn: -0.8399957	total: 1m 26s	remaining: 23.2s
630:	learn: -0.8399002	total: 1m 26s	remaining: 23.1s
631:	learn: -0.8398546	total: 1m 26s	remaining: 23s
632:	learn: -0.8398179	total: 1m 26s	remaining: 22.8s
633:	learn: -0.8397348	total: 1m 26s	remaining: 22.7s
634:	learn: -0.8396791	total: 1m 26s	remaining: 22.6s
635:	learn: -0.8396548	total: 1m 26s	remaining: 22.4s
636:	learn: -0.8396196	total: 1m 27s	remaining: 22.3s
637:	learn: -0.8395945	total: 1m 27s	remaining: 22.2s
638:	learn: -0.8395301	total: 1m 27s	remaining: 22s
639:	learn: -0.8394411	total: 1m 2

775:	learn: -0.8328205	total: 1m 47s	remaining: 3.31s
776:	learn: -0.8327663	total: 1m 47s	remaining: 3.17s
777:	learn: -0.8327079	total: 1m 47s	remaining: 3.04s
778:	learn: -0.8326295	total: 1m 47s	remaining: 2.9s
779:	learn: -0.8324949	total: 1m 47s	remaining: 2.76s
780:	learn: -0.8324432	total: 1m 47s	remaining: 2.62s
781:	learn: -0.8324076	total: 1m 47s	remaining: 2.48s
782:	learn: -0.8323436	total: 1m 48s	remaining: 2.35s
783:	learn: -0.8322906	total: 1m 48s	remaining: 2.21s
784:	learn: -0.8322506	total: 1m 48s	remaining: 2.07s
785:	learn: -0.8321909	total: 1m 48s	remaining: 1.93s
786:	learn: -0.8321634	total: 1m 48s	remaining: 1.79s
787:	learn: -0.8321277	total: 1m 48s	remaining: 1.66s
788:	learn: -0.8321135	total: 1m 48s	remaining: 1.52s
789:	learn: -0.8320316	total: 1m 49s	remaining: 1.38s
790:	learn: -0.8319632	total: 1m 49s	remaining: 1.24s
791:	learn: -0.8319192	total: 1m 49s	remaining: 1.1s
792:	learn: -0.8318776	total: 1m 49s	remaining: 967ms
793:	learn: -0.8318328	total: 

BayesSearchCV(cv=3, error_score='raise',
       estimator=<catboost.core.CatBoostClassifier object at 0x11674bc50>,
       fit_params=None, iid=True, n_iter=10, n_jobs=-1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False, scoring='neg_log_loss',
       search_spaces={'one_hot_max_size': [24], 'learning_rate': [0.01, 1], 'loss_function': ['MultiClass'], 'depth': [6, 10], 'n_estimators': [800]},
       verbose=True)

In [54]:
bs.best_score_

-0.8682877193318085

In [55]:
bs.best_params_

{'depth': 8,
 'learning_rate': 0.022944294136017482,
 'loss_function': 'MultiClass',
 'n_estimators': 800,
 'one_hot_max_size': 24}

In [ ]:
cat_boost = CatBoostClassifier(cat_features=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
                               depth=8, learning_rate = .)